In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# ============================================================
# 📁 PATHS
# ============================================================
BASE_DIR = "/data/ephemeral/home/data"
IN_META = os.path.join(BASE_DIR, "meta_stage0_10_0_train_v10.csv")
IN_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_0_train_v10")

OUT_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_1_train_v10")
OUT_META = os.path.join(BASE_DIR, "meta_stage0_10_1_train_v10.csv")

os.makedirs(OUT_DIR, exist_ok=True)

# ============================================================
# ⚙️ 전처리 함수
# ============================================================

def normalize_background(img):
    """문서 대비 강화 + 배경 평탄화"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bg = cv2.GaussianBlur(gray, (55, 55), 0)
    norm = cv2.divide(gray, bg, scale=255)
    norm = cv2.merge([norm, norm, norm])
    return np.clip(norm, 0, 255).astype(np.uint8)

def white_balance(img):
    """화이트 밸런스 (Gray World)"""
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(result)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    result = cv2.merge((l, a, b))
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

def resize_and_pad(img, size=768, pad_color=(255, 255, 255)):
    """비율 유지 리사이즈 후 흰색 패딩"""
    h, w = img.shape[:2]
    s = size / max(h, w)
    nh, nw = int(h * s), int(w * s)
    img = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA if s < 1 else cv2.INTER_CUBIC)
    top = (size - nh) // 2
    bottom = size - nh - top
    left = (size - nw) // 2
    right = size - nw - left
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=pad_color)

def preprocess(img):
    img = normalize_background(img)
    img = white_balance(img)
    img = resize_and_pad(img, size=768)
    return img

# ============================================================
# 🚀 전처리 실행
# ============================================================
meta = pd.read_csv(IN_META)
results = []
fail = 0

print(f"🧩 전처리 시작: {len(meta)}장")
for i, row in tqdm(meta.iterrows(), total=len(meta)):
    path = row["filepath"]
    target = int(row["target"])
    fname = os.path.basename(path)
    img = cv2.imread(path)
    if img is None:
        fail += 1
        continue

    try:
        proc = preprocess(img)
        save_path = os.path.join(OUT_DIR, fname)
        cv2.imwrite(save_path, proc)
        results.append((save_path, fname, target))
    except Exception as e:
        fail += 1
        print(f"[!] {fname} 처리 실패: {e}")

# ============================================================
# 💾 메타 저장
# ============================================================
meta_out = pd.DataFrame(results, columns=["filepath", "basename", "target"])
meta_out.to_csv(OUT_META, index=False)

print(f"\n✅ 완료: {len(meta_out)}장 저장 (실패 {fail}장)")
print(f"📁 출력 경로: {OUT_DIR}")
print(f"📄 메타 파일: {OUT_META}")


🧩 전처리 시작: 100480장


100%|██████████████████████| 100480/100480 [13:14<00:00, 126.52it/s]



✅ 완료: 100480장 저장 (실패 0장)
📁 출력 경로: /data/ephemeral/home/data/processed/stage0_10_1_train_v10
📄 메타 파일: /data/ephemeral/home/data/meta_stage0_10_1_train_v10.csv


In [2]:
# ============================================================
# 📁 PATHS
# ============================================================
BASE_DIR = "/data/ephemeral/home/data"

IN_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_0_test_v10")  # 증강된 test 이미지 폴더
OUT_DIR = os.path.join(BASE_DIR, "processed", "stage0_10_1_test_v10")
OUT_META = os.path.join(BASE_DIR, "meta_stage0_10_1_test_v10.csv")

RAW_TEST_DIR = os.path.join(BASE_DIR, "raw", "test")
SAMPLE_SUB_PATH = os.path.join(BASE_DIR, "raw", "sample_submission.csv")

os.makedirs(OUT_DIR, exist_ok=True)

# ============================================================
# ⚙️ 전처리 함수 (train과 동일)
# ============================================================

def normalize_background(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bg = cv2.GaussianBlur(gray, (55, 55), 0)
    norm = cv2.divide(gray, bg, scale=255)
    norm = cv2.merge([norm, norm, norm])
    return np.clip(norm, 0, 255).astype(np.uint8)

def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(result)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    result = cv2.merge((l, a, b))
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

def resize_and_pad(img, size=768, pad_color=(255, 255, 255)):
    h, w = img.shape[:2]
    s = size / max(h, w)
    nh, nw = int(h * s), int(w * s)
    img = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA if s < 1 else cv2.INTER_CUBIC)
    top = (size - nh) // 2
    bottom = size - nh - top
    left = (size - nw) // 2
    right = size - nw - left
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=pad_color)

def preprocess(img):
    img = normalize_background(img)
    img = white_balance(img)
    img = resize_and_pad(img, size=768)
    return img


# ============================================================
# 🚀 전처리 실행 (sample_submission 기준으로 ID 탐색)
# ============================================================
sample = pd.read_csv(SAMPLE_SUB_PATH)
test_ids = sample["ID"].apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")

results = []
fail = 0

print(f"🧩 Test 전처리 시작: {len(test_ids)}장")
for img_name in tqdm(test_ids):
    path = os.path.join(RAW_TEST_DIR, img_name)
    if not os.path.exists(path):
        fail += 1
        continue

    img = cv2.imread(path)
    if img is None:
        fail += 1
        continue

    try:
        proc = preprocess(img)
        save_path = os.path.join(OUT_DIR, img_name)
        cv2.imwrite(save_path, proc)
        results.append((save_path, img_name))
    except Exception as e:
        fail += 1
        print(f"[!] {img_name} 처리 실패: {e}")

# ============================================================
# 💾 메타 저장
# ============================================================
meta_out = pd.DataFrame(results, columns=["filepath", "basename"])
meta_out.to_csv(OUT_META, index=False)

print(f"\n✅ 완료: {len(meta_out)}장 저장 (실패 {fail}장)")
print(f"📁 출력 경로: {OUT_DIR}")
print(f"📄 메타 파일: {OUT_META}")


🧩 Test 전처리 시작: 3140장


100%|██████████████████████████| 3140/3140 [00:22<00:00, 138.35it/s]


✅ 완료: 3140장 저장 (실패 0장)
📁 출력 경로: /data/ephemeral/home/data/processed/stage0_10_1_test_v10
📄 메타 파일: /data/ephemeral/home/data/meta_stage0_10_1_test_v10.csv
